# 아영 - Story Generator

### Note
기본 Ko GPT2

-> 동화 Fine Tuning

-> 확률 필터링

- 아래 결과는 epoch:230, loss:0.02xx 인 학습모델 사용

# 1. Ready

## 1.1 기본 패키지 임포트

In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel


## 1.2 학습 모델 다운로드

In [2]:
MODEL_NAME = "skt/kogpt2-base-v2"

### 토크나이저 load

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)

TOKENS_DICT = {
    'bos_token':'<s>',
    'eos_token':'</s>',
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'mask_token':'<mask>'
}

# 특수 토큰이 토크나이저에 추가되고 모델은 수정된 토크나이저에 맞게 임베딩의 크기를 조정
tokenizer.add_special_tokens(TOKENS_DICT)

print(tokenizer.special_tokens_map)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


### 미세조정학습 모델 load

In [4]:
from util.model import model_loading as load

In [8]:
checkpointPath = "./modelCheckpoint/checkpointmodel4.tar"
loading = True

In [9]:
model,ckp = load(checkpointPath, PU = 'cpu', status = loading)

In [10]:
print(ckp['epoch'], ckp['loss'])

450 tensor(0.0245, requires_grad=True)


# 2. Generate

In [23]:
from util.generator import sample_sequence as gen

In [12]:
context = "소년이 섬에서 노인을 만났다."

In [13]:
generated = generate(model, length=200, context=context, num_samples=1, tokenizer=tokenizer)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:46<00:00,  4.26it/s]


In [14]:
print(tokenizer.decode(generated[0]))

소년이 섬에서 노인을 만났다. 
노인은 시내로 가  "드디어도 주지 못하였사옵니다." 라고 말했다. 
 "저도 모르겠사옵니다." 라며 스승님이 말했다.  "만약 네가 형을 사과를 선물로 받쳤다면 합니다." 
 "그럼 어디 한 번 약속해주시오?" 라며 주인이 말했다. 
 "사냥꾼 빅맨 7권." 
 "원, 그런 거 어디 있냐?" 라며 주인장이 말했다.  "수습 젊은이이자면서요. 하지만 누구도 수염은 어찌나 닮던지요!
" 보고 싶음, 내리트가 말을 걸었다. 
 "그거라면 우리밖에 없을 거다." 라며 사냥꾼이 말했다.  "내 용무쟁이가 있거든, 내 형들도 할  없네요." 
사냥꾼은 다만 그동안 있었던 일을 죄다 왕께 고해바쳤다. 
적들은 이번에도 왕께 나서를 보냈다.
그런 다음 젊은이를 본 그들은 말했다. 
 "오늘밤 즉시 여기로 사람을


# Ko-GPT2 파인튜닝 비교

- Input Context : 별 하나의 추억과, 별 하나의 사랑과

## (1)파인튜닝 전

In [40]:
import torch
from transformers import GPT2LMHeadModel

model_base = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '길을 잃은 소녀를 말에 태운 왕자는 '
input_ids = tokenizer.encode(text)
gen_ids = model_base.generate(torch.tensor([input_ids]),
                            max_length=128,
                            repetition_penalty=2.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

길을 잃은 소녀를 말에 태운 왕자는 쏜살같이 달려와 그를 구해냈다.
그런데 그 순간, 갑자기 나타난 괴한들이 그녀를 향해 총을 겨누고 있었다.
"이봐! 이게 누구야?"
괴한은 그녀의 몸을 노려보며 소리쳤다.
그리고는 재빨리 도망치기 시작했다.
하지만 그녀는 이미 죽은 목숨이었다.
왕자가 총에 맞은 것은 바로 그때였다.
그는 다시 한 번 칼을 휘둘러 그의 목을 베었다.
그러나 그는 곧 정신을 차리고 말았다.
순간적으로 일어난 일이었다.
그의 몸은 순식간에 아수라장이 되어버렸다.
아무리 생각해도 끔찍했다.
'저놈은 저렇게 죽어도 상관없어.


- 기본적으로 수필/에세이 느낌이 남 ex) ~이다. ~다.
- 대화체가 잘 없음
- 문장부호 잘 캐치함

## (2)파인튜닝 후

In [41]:
import torch
from transformers import GPT2LMHeadModel

text = '길을 잃은 소녀를 말에 태운 왕자는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                            max_length=128,
                            repetition_penalty=2.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

길을 잃은 소녀를 말에 태운 왕자는 그래도 자신의 발을 들여 아가씨가 다시 거닐 수 있었어요.
그러자 처녀가 말했어요.
 "그이는 어디 가고, 저는 왕자님을 데리고 있는데?" 
왕자가 워낙 낯설기 시작한지라 그 여자도 이 말을 듣곤 나가버렸지요.
하지만 그녀는 자신이 예전에 구해준 반지를 기억해내려고요. 하며 멀리 가지 않을 뿐이었 있으니까요.
마침내 그녀 엄마가 왕궁에 도착하고 나는 딸을 결혼식에 초대했어요, 엄마. 딸라면 내가 그와 함께 온다며 버찌 사을 먹었 후 그릇에 가득 담아왔어.
이 지역은 왕의 따님


- 동화 데이터를 학습한 후 동화에서 사용되는 어미등장 ex)~어요, ~답니다, ~지요
- 대화체를 제대로 학습함
- 따옴표의 사용과 쉼표의 사용이 적절함

## (3)확률 필터링 후

In [45]:

generated = gen(model, 
                     length=200, 
                     context='길을 잃은 소녀를 말에 태운 왕자는',
                     num_samples=1, 
                     repetition_penalty=2.0,
                     top_p=0.9,
                     tokenizer = tokenizer)
print(tokenizer.decode(generated[0]))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:47<00:00,  4.20it/s]

길을 잃은 소녀를 말에 태운 왕자는 자신의 말은 타고 성큼 빠른 속도로 사라져버렸지요.
그러자 요정들이 달려가 그녀를 뒤쫓아 왔지요, 하지만 아무도 그녀도 발각되지 못했답니다.
소녀는 공주와 함께 위로 올라갔어요. 그런데 한 번도 본 적이 없는 땅속 밑에서 오래된 풀 냄새로 찾아내려는 게 아니라 넓고 언덕에서 아래로 내려오는 보트 세 개를 끌고 있었어요.
그녀가 한때 그렇게 물었죠.
 "저," 라며 그녀가 대답했죠.  "이 상자 안에 누구 들어 올려다 볼 수 없어요! 문 뒤에 내려앉세련." 
공주가 말했어요.
" 치켜든 채로 들어왔다. 왜냐면 들여다보던 공이 너무 위층으로 올라가 그만 용기를 잃고 뚜껑이 덮인 작은 유리 산에 도착하고 엉엉한 작고 낡은 남자가 거기 있다 갇히고 자고 했다.신당 뭐가 운 길인가요?
황소가 여우를 부르더니 그 안에서 그를 풀어주었다. 순무는 마차 만드는 소리가 아니라, 죽은 듯이 납작 엎드려있어서 마치 산토끼처럼 바닥을 드러누웠어.
여우는 바람에 머리카락이 날


- 위와 더불어 전체를 관통하는 일관된 키워드가 있음  ex)눈물
- 여기서 키워드적용을 할 수 있을까?

In [44]:
generated = gen(model, 
                     length=200, 
                     context='나 너 좋아하냐?' , 
                     num_samples=1, 
                     repetition_penalty=2.0,
                     top_p=0.9,
                     tokenizer = tokenizer)
print(tokenizer.decode(generated[0]))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:47<00:00,  4.25it/s]

나 너 좋아하냐? 제가 가만데버릇 남 주 찾아나서 저러 데려가 달라" 
 "심술궂은 패거리." 
내가 손가락으로 여인숙 주인장을 가리키며 말했어요.
하지만 나는 그가 어디 있는지 말하지 않고 거기서 기다리겠다고 우겨 맞는 꼴을 도저히 찾지 못하고 맙소연이었어요. 아니, 꼭 욕심을 부리려하는 수밖에 없는 걸요.
받아먹긴 최고의 도둑이 아니었다면 난 잠시라도 우리와 내 영혼을 바꿀 수 없을 거예요, 그게 행운이 찾아와줬으면 좋겠습니다.
이게 뭐지?
우리는 양배추 백과와 물고기를 잡은 다음 시골여관으로 가 푸주한 손수레를 끌고 왔어요.
포도색 담벼락에 걸린 현수막이 나오자, 오리가 한 마리씩 튀었습니다 돼요. 송아지는 내가 예전에 만나던 길고양이 말이 곱해져 이제 곧장 개울가로 심부름을 오게 되었답니다.
그가 자신은 요리를 멈추고 서서는 소릴 질렀다고 했지요.
내 말을 듣고 거짓말쟁이 농부는 벌
